In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

URL = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet"
df = pd.read_parquet(URL)

print(f"Number of records: {len(df)}")


Number of records: 3403766


In [3]:
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

df['duration'] = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime).dt.total_seconds() / 60
df = df[(df.duration >= 1) & (df.duration <= 60)]

df[['PULocationID', 'DOLocationID']] = df[['PULocationID', 'DOLocationID']].astype(str)

print(f"Filtered records: {len(df)}")


Filtered records: 3316216


In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

categorical = ['PULocationID', 'DOLocationID']
train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
y_train = df['duration'].values

model = LinearRegression()
model.fit(X_train, y_train)

print(f"Model intercept: {round(model.intercept_, 2)}")


Model intercept: 24.77


In [6]:
pip install mlflow

  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
   ---------------------------------------- 28.2/28.2 MB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 6.1/6.1 MB 6.2 MB/s eta 0:00:00
   ---------------------------------------- 8.1/8.1 MB 7.3 MB/s eta 0:00:00
   ---------------------------------------- 681.0/681.0 kB 8.9 MB/s eta 0:00:00
   ---------------------------------------- 2.2/2.2 MB 8.3 MB/s eta 0:00:00
Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)
Using cached werkzeug-3.1.3-py3-none-any.whl (224 kB)
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.0.3
    Uninstalling Werkzeug-3.0.3:
      Successfully uninstalled Werkzeug-3.0.3
  Attempting uninstall: itsdangerous
    Found existing installation: itsdangerous 1.1.0
    Uninstalling itsdangerous-1.1.0:
      Successfully uninstalled itsdangerous-1.1.0
Note: you may need to restart the

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mage-ai 0.9.75 requires itsdangerous~=1.1.0, but you have itsdangerous 2.2.0 which is incompatible.
mage-ai 0.9.75 requires Werkzeug==3.0.3, but you have werkzeug 3.1.3 which is incompatible.


In [8]:
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
import pickle
import pandas as pd

# Load and prepare data
df = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet")
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
df['duration'] = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime).dt.total_seconds() / 60
df = df[(df.duration >= 1) & (df.duration <= 60)]
df[['PULocationID', 'DOLocationID']] = df[['PULocationID', 'DOLocationID']].astype(str)

# Features and labels
categorical = ['PULocationID', 'DOLocationID']
train_dicts = df[categorical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
y_train = df['duration'].values

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Set MLflow tracking
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("nyc-taxi-experiment")

with mlflow.start_run():
    # Log model
    mlflow.sklearn.log_model(model, "model", registered_model_name="lin_reg_model_hw3")

    # Log DictVectorizer as artifact
    with open("dv.pkl", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("dv.pkl")

    print("✅ Model and DictVectorizer logged and registered.")


2025/03/26 12:51:43 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.
2025/03/26 12:51:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'lin_reg_model_hw3'.
2025/03/26 12:51:50 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: lin_reg_model_hw3, version 1


✅ Model and DictVectorizer logged and registered.
🏃 View run nervous-horse-966 at: http://localhost:5000/#/experiments/1/runs/b80e07b34cc040e28bc2de9db27ba62f
🧪 View experiment at: http://localhost:5000/#/experiments/1


Created version '1' of model 'lin_reg_model_hw3'.
